In [1]:
# Import Libraries
import keras
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from IPython.display import FileLink, FileLinks
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import pylab
from scipy.stats.stats import pearsonr

Using TensorFlow backend.


In [2]:
### Choose one file, make cuts, and print the number of samples to train on

datafilenames = ["SharedHits/NormalizedCharge/output_final_1.h5","SharedHits/AbsoluteCharge/output_final_1.h5",
                "Shared100NonShared/NormalizedCharge/output_final_1.h5","Shared100NonShared/AbsoluteCharge/output_final_1.h5",
                "AllHits/NormalizedCharge/output_final_1.h5","AllHits/AbsoluteCharge/output_final_1.h5"]

counter = 5

datafilename = datafilenames[counter]

numentries = 10000

# Import the data
df = pd.read_hdf("/uscms_data/d3/bbonham/TrackingDstar/LambaAnalyzer/output_of_postprocess/"+datafilename, key="df", mode='r')

# Setup different cuts for different data
if counter == 0 or counter == 1: # Shared Hits
    cuttitle = "nUniqueSimTracksInSharedHit >= 0"
    stringforcut = "(df['nUniqueSimTracksInSharedHit']>=0)"
if counter == 2 or counter == 3: # Shared100NonShared Hits
    cuttitle = "nUniqueSimTracksInSharedHit != (0|1)"
    stringforcut = '(df["nUniqueSimTracksInSharedHit"]!=0) & (df["nUniqueSimTracksInSharedHit"]!=1)'
if counter == 4 or counter == 5: # All Hits
    cuttitle = "nUniqueSimTracksInSharedHit != (0|1)"
    stringforcut = '(df["nUniqueSimTracksInSharedHit"]!=0) & (df["nUniqueSimTracksInSharedHit"]!=1)'

# Print info on dataframe
print ""
print '\033[1m' + datafilename[:-18] + '\033[0m' # name printed in bold
print df.info()
print ""

## Make Cuts ##

# Compute the pixel_number variable
pixelColumns = ["pixel_%i" % x for x in range(400)]
pixels_df = df[pixelColumns].values
pixel_number = pixels_df.astype(bool).sum(axis=1)
df.insert(0, "Pixel_number", pixel_number)

# Make final cuts
df = df[df["Pixel_number"]>1] # ensure > 1 pixel is involved
df = df[(df["GenDeltaR"]<0.1)]
df = df[eval(stringforcut)]

# Print info on dataframe
print '\033[1m' + datafilename[:-18] + '\033[0m' # name printed in bold
print df.info()
print ""

IOError: File /uscms_data/d3/bbonham/TrackingDstar/LambaAnalyzer/output_of_postprocess/AllHits/AbsoluteCharge/output_final_1.h5 does not exist

In [ ]:
print round(df.memory_usage().sum()/1024./1024.,1),"MB"
print "train on",int(round((df.shape[0]/2.)*.9)),"samples"
print "validate on",int(round((df.shape[0]/2.)*.1)),"samples"